# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from sklearn.feature_extraction.text import (
    CountVectorizer
)
from sklearn.feature_selection import (
    SelectKBest, SelectPercentile,
    SelectFpr, VarianceThreshold,
    SelectFromModel,
    chi2, f_classif,
)
from sklearn.ensemble import (
    RandomForestClassifier
)
from sklearn.linear_model import (
    LogisticRegression
)
from sklearn.pipeline import (
    FeatureUnion, Pipeline
)
from scipy.sparse import hstack

from sklearn.naive_bayes import BernoulliNB

%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
display(
    X_train.isnull().sum()[X_train.isnull().sum() > 0],
    X_test.isnull().sum()[X_test.isnull().sum() > 0]
)

funder                3635
installer             3655
subvillage             371
public_meeting        3334
scheme_management     3877
scheme_name          28166
permit                3056
dtype: int64

funder                869
installer             877
subvillage             99
public_meeting        821
scheme_management     969
scheme_name          7092
permit                737
dtype: int64

In [5]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [6]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [7]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

X_train.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_train.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))
X_test.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_test.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))


X_train.loc[:, 'date_recorded_month'] /= 12
X_test.loc[:, 'date_recorded_month'] /= 12

In [8]:
X_test.loc[:,'date_recorded_year'].fillna(0, inplace=True)
X_train.loc[:,'date_recorded_year'].replace([0, np.nan], X_test.loc[:,'date_recorded_year'].median(), inplace=True)
X_test.loc[:,'date_recorded_year'].replace([0, np.nan], X_test.loc[:,'date_recorded_year'].median(), inplace=True)
X_test.loc[:,'construction_year'].fillna(0, inplace=True)
X_train.loc[:,'construction_year'].replace([0, np.nan], X_test.loc[:,'construction_year'].median(), inplace=True)
X_test.loc[:,'construction_year'].replace([0, np.nan], X_test.loc[:,'construction_year'].median(), inplace=True)

In [9]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [10]:
X_train.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)
X_test.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)

In [11]:
heightscale = StandardScaler().fit(pd.concat((X_train.loc[:, 'gps_height'],
                                              X_test.loc[:, 'gps_height']), ignore_index=True))
X_train.loc[:, 'gps_height'] = heightscale.transform(X_train.loc[:, 'gps_height']).reshape(-1, 1)
X_test.loc[:, 'gps_height'] = heightscale.transform(X_test.loc[:, 'gps_height']).reshape(-1, 1)

/home/god/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-pack

In [12]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population'])
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population'])
X_train.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_test.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_train.loc[:,'amount_per_man'] /= X_test.loc[:,'amount_per_man'].max()
X_test.loc[:,'amount_per_man'] /= X_test.loc[:,'amount_per_man'].max()
X_train.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_test.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)

X_train.loc[:,'population'] /= X_test.loc[:,'population'].max()
X_test.loc[:,'population'] /= X_test.loc[:,'population'].max()

# display(X_test.loc[:,'amount_per_man'])
#amntpmscaler = StandardScaler().fit(X_test.loc[:, 'amount_per_man'])
#X_train.loc[:,'amount_per_man'] = amntpmscaler.transform(X_train.loc[:,'amount_per_man']).reshape(-1, 1)
#X_test.loc[:,'amount_per_man'] = amntpmscaler.transform(X_test.loc[:,'amount_per_man']).reshape(-1, 1)

In [13]:
X_train.loc[:,'gps_height'].replace([np.inf, -np.inf, np.nan, 0],
                                    X_train.loc[:,'gps_height'].median(), inplace=True)
X_test.loc[:,'gps_height'].replace([np.inf, -np.inf, np.nan, 0],
                                    X_train.loc[:,'gps_height'].median(), inplace=True)

In [14]:
amounttshscaler = StandardScaler().fit(pd.concat((X_train.loc[:, 'amount_tsh'],
                                              X_test.loc[:, 'amount_tsh']), ignore_index=True))
X_train.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_train.loc[:, 'amount_tsh']).reshape(-1, 1)
X_test.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_test.loc[:, 'amount_tsh']).reshape(-1, 1)

/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/.venv/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single 

In [15]:
X_train.loc[:,'age'].fillna(X_test.loc[:,'age'].median(), inplace=True)
X_test.loc[:,'age'].fillna(X_test.loc[:,'age'].median(), inplace=True)

In [16]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [17]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [18]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [19]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [20]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [21]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [22]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [23]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [24]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [25]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [26]:
drop_cats = ['source_type', 'source_class',
             'water_quality', 'quality_group',
             'quantity_group', 'waterpoint_type_group',
             'payment_type', 'extraction_type_group',
             'extraction_type_class', 'management_group',
             'construction_year', 'date_recorded_month'
]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [27]:
X_train.iloc[0,:]

amount_tsh                                         1.95453
funder                                               Roman
gps_height                                         1.04558
installer                                            Roman
longitude                                          34.9381
latitude                                          -9.85632
wpt_name                                                 1
basin                                           Lake Nyasa
subvillage                                        Mnyusi B
region                                              Iringa
region_code                                             11
district_code                                            5
lga                                                 Ludewa
ward                                              Mundindi
population                                      0.00950388
public_meeting                                           1
scheme_management                                      V

In [28]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []
text_selfeat_names = []

In [29]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [30]:
# lonlat shall be clusterized before usage
# day probably doesn't matter, but month might be (seasonality)
# the years in test and train do not intersect
X_train.drop(['longitude', 'latitude',
              'date_recorded_day', 'date_recorded_year',
              'installer', 'funder', 'subvillage', 'wpt_name'], axis=1, inplace=True, errors='ignore')
X_test.drop(['longitude', 'latitude',
              'date_recorded_day', 'date_recorded_year',
              'installer', 'subvillage', 'wpt_name'], axis=1, inplace=True, errors='ignore')

In [31]:
display(X_train.shape, X_test.shape)

(59400, 24)

(14850, 25)

In [32]:
sel = Pipeline([
    ('varthres', VarianceThreshold(1e-5)),
    ('chi2', SelectFpr(chi2, 1e-24)),
    #('interact', PolynomialFeatures(2, interaction_only=True)),
    #('feats', FeatureUnion([
        # ('anova', SelectFpr(f_classif, 1e-19)), # yeah, i'm gonna be very pretentious!
        # ('chi2', SelectKBest(chi2, min(128, tr.shape[1])).fit(tr, ytr),)
        # OHE-encoded features are supposed to have additive impact, so linear model:
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=100, max_depth=30)))
    #])),
])
for cat in text_features:
    if cat not in X_train.columns:
        continue
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = CountVectorizer(ngram_range=(1,2), binary=True)
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    #kbest = SelectKBest(chi2, min(8, tr.shape[1])).fit(tr, ytr)

    sel.fit(tr, ytr)
    tr = sel.transform(tr)
    te = sel.transform(te)
    for i in range(tr.shape[1]):
        text_selfeat_names.append('%s_%s' % (cat, i))
    #tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    #te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    if tr.shape[1] > 0:
        text_feats_train.append(tr)
        text_feats_test.append(te)

text_feats_train = hstack(text_feats_train) # pd.DataFrame(hstack(text_feats_train), columns=text_selfeat_names)
text_feats_test = hstack(text_feats_test) # pd.DataFrame(hstack(text_feats_test), columns=text_selfeat_names)
# print(sel.fit_transform(text_feats_train, ytr).toarray().shape)
# sel_text_feats = SelectKBest(chi2, 48).fit(text_feats_train, ytr) # SelectFpr(f_classif, 1e-19)
sel_text_feats = sel
text_feats_train = sel.fit_transform(text_feats_train, ytr)
text_feats_test = sel.transform(text_feats_test)
print('cat features shape: ', text_feats_train.shape)
# print('top catfeats: ', sel.named_steps['linear'].estimator_.coef_[:20] )
del tr, te

X_train = X_train.drop(text_features, axis=1, errors='ignore')
cat_features = np.array(list(range(X_train.shape[1], # + berntr.shape[1],
                          X_train.shape[1] + text_feats_train.shape[1]))) # + berntr.shape[1] ))
X_train = pd.concat([X_train] +
                    [pd.DataFrame(text_feats_train.toarray())],
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore')] +
                   [pd.DataFrame(text_feats_test.toarray())],
                    axis=1)

cat features shape:  (59400, 26)


In [33]:
X_train.columns

Index([    'amount_tsh',     'gps_height',     'population', 'public_meeting',
               'permit',     'dry_season',   'rainy_season',            'age',
       'amount_per_man',                0,                1,                2,
                      3,                4,                5,                6,
                      7,                8,                9,               10,
                     11,               12,               13,               14,
                     15,               16,               17,               18,
                     19,               20,               21,               22,
                     23,               24,               25],
      dtype='object')

In [34]:
X_train.head()

,amount_tsh,gps_height,population,public_meeting,permit,dry_season,rainy_season,age,amount_per_man,0,...,16,17,18,19,20,21,22,23,24,25
0,1.95452941861,1.04558069781,0.00950388002441,1.0,0.0,1,0,12,0.00183486238532,1,...,0,0,0,0,0,0,0,0,1,0
1,-0.109636705889,1.05857227813,0.02441363676,0.5,1.0,1,0,3,0.0,1,...,1,1,0,0,0,0,1,0,1,0
2,-0.101036013704,0.0293504149544,0.0217978899643,1.0,1.0,1,0,4,3.33333333333e-06,0,...,0,0,0,0,0,0,1,0,0,0
3,-0.109636705889,-0.581253860116,0.00505711047171,1.0,1.0,1,0,27,0.0,0,...,1,1,0,0,1,1,0,0,0,0
4,-0.109636705889,-0.960896707264,0.0,1.0,1.0,0,0,25,0.0,1,...,1,1,0,0,0,0,1,0,1,0


In [35]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('dry_season', dtype('O')),
 ('rainy_season', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [36]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)

In [37]:
bern = BernoulliNB()
bern.fit(xtr.iloc[:, cat_features], ytr)
logit = LogisticRegression().fit(xtr.iloc[:, cat_features], ytr)

xtr = pd.concat([xtr.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xtr.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xtr.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

xcv = pd.concat([xcv.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xcv.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xcv.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

xva = pd.concat([xva.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xva.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xva.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

xte = pd.concat([xte.reset_index(drop=True),
                 pd.DataFrame(bern.predict_proba(xte.iloc[:, cat_features]))
                 .add_prefix('p_').reset_index(drop=True),
                 pd.DataFrame(logit.predict_proba(xte.iloc[:, cat_features]))
                 .add_prefix('li_').reset_index(drop=True),],
                axis=1)

In [38]:
xva.head()

,amount_tsh,gps_height,population,public_meeting,permit,dry_season,rainy_season,age,amount_per_man,0,...,22,23,24,25,p_0,p_1,p_2,li_0,li_1,li_2
0,-0.0408311684062,-0.55382719055,0.0479553579214,1.0,1.0,1,0,2,1.21212121212e-05,0,...,1,0,1,0,0.857666,0.122223,0.020112,0.675992,0.175021,0.148987
1,-0.109636705889,-0.221820137912,0.00348766239428,1.0,1.0,1,0,23,0.0,1,...,0,0,1,0,0.650067,0.149172,0.200760,0.762427,0.061477,0.176095
2,-0.109636705889,-0.960896707264,0.0,1.0,1.0,0,0,25,0.0,1,...,0,1,0,0,0.446096,0.045028,0.508876,0.550184,0.086015,0.363802
3,-0.109636705889,-0.960896707264,0.0,0.5,1.0,0,0,25,0.0,1,...,0,1,0,0,0.356943,0.029671,0.613386,0.583756,0.058834,0.357410
4,-0.109636705889,-0.960896707264,0.0,1.0,1.0,0,1,26,0.0,1,...,0,0,1,0,0.359842,0.058210,0.581948,0.463485,0.051332,0.485184


In [39]:
display(xtr.shape, xte.shape)

(41578, 41)

(14850, 41)

In [40]:
#X_train.to_csv('X_train_prepr2.csv')
#X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
#pd.Series(ytr).to_csv('y_train_prepr2.csv')
#pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [41]:
import catboost

In [42]:
clf = catboost.CatBoostClassifier(500, loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importance=True)

In [43]:
#clf.fit(xtr, ytr, cat_features=cat_features,
#        eval_set=(xcv, ycv), verbose=True)

In [44]:
#clf.score(xcv, ycv)

In [45]:
#clf.score(xva, yva)

In [46]:
y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)

ytr2 = y_bin.transform(ytr)
ycv2 = y_bin.transform(ycv)
yva2 = y_bin.transform(yva)

xscaler = StandardScaler().fit(xte)
xtr2 = xscaler.transform(xtr)
xte2 = xscaler.transform(xte)
xcv2 = xscaler.transform(xcv)
xva2 = xscaler.transform(xva)

def y_to_df(y, ids):
    y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df


In [47]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.utils

Using TensorFlow backend.


In [ ]:
clf = Sequential([
    Dense(2*xtr.shape[1], input_shape=(xtr.shape[1],), activation='softmax'),
    Dropout(.1),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(3, activation='softmax'),
])
clf.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
clf.fit(xtr2, ytr2,
      batch_size=256,
      epochs=3000)


Epoch 1/3000
41578/41578 [==============================] - 0s - loss: 1.0240 - acc: 0.4963     
Epoch 2/3000
41578/41578 [==============================] - 0s - loss: 0.9230 - acc: 0.5431     
Epoch 3/3000
41578/41578 [==============================] - 0s - loss: 0.8642 - acc: 0.5643     
Epoch 4/3000
41578/41578 [==============================] - 0s - loss: 0.8195 - acc: 0.6967     
Epoch 5/3000
41578/41578 [==============================] - 0s - loss: 0.7805 - acc: 0.7087     
Epoch 6/3000
41578/41578 [==============================] - 0s - loss: 0.7511 - acc: 0.7144     
Epoch 7/3000
41578/41578 [==============================] - 0s - loss: 0.7293 - acc: 0.7258     
Epoch 8/3000
41578/41578 [==============================] - 0s - loss: 0.7126 - acc: 0.7267     
Epoch 9/3000
41578/41578 [==============================] - 0s - loss: 0.7025 - acc: 0.7289     
Epoch 10/3000
41578/41578 [==============================] - 0s - loss: 0.6941 - acc: 0.7314     
Epoch 11/3000
41578/41578 [==

41578/41578 [==============================] - 0s - loss: 0.5895 - acc: 0.7607     
Epoch 85/3000
41578/41578 [==============================] - 0s - loss: 0.5889 - acc: 0.7606     
Epoch 86/3000
41578/41578 [==============================] - 0s - loss: 0.5879 - acc: 0.7607     
Epoch 87/3000
41578/41578 [==============================] - 0s - loss: 0.5861 - acc: 0.7609     
Epoch 88/3000
41578/41578 [==============================] - 0s - loss: 0.5879 - acc: 0.7615     
Epoch 89/3000
41578/41578 [==============================] - 0s - loss: 0.5876 - acc: 0.7606     
Epoch 90/3000
41578/41578 [==============================] - 0s - loss: 0.5878 - acc: 0.7608     
Epoch 91/3000
41578/41578 [==============================] - 0s - loss: 0.5869 - acc: 0.7628     
Epoch 92/3000
41578/41578 [==============================] - 0s - loss: 0.5870 - acc: 0.7605     
Epoch 93/3000
41578/41578 [==============================] - 0s - loss: 0.5864 - acc: 0.7623     
Epoch 94/3000
41578/41578 [=======

41578/41578 [==============================] - 0s - loss: 0.5716 - acc: 0.7656     
Epoch 168/3000
41578/41578 [==============================] - 0s - loss: 0.5692 - acc: 0.7672     
Epoch 169/3000
41578/41578 [==============================] - 0s - loss: 0.5700 - acc: 0.7671     
Epoch 170/3000
41578/41578 [==============================] - 0s - loss: 0.5697 - acc: 0.7670     
Epoch 171/3000
41578/41578 [==============================] - 0s - loss: 0.5705 - acc: 0.7676     
Epoch 172/3000
41578/41578 [==============================] - 0s - loss: 0.5700 - acc: 0.7665     
Epoch 173/3000
41578/41578 [==============================] - 0s - loss: 0.5699 - acc: 0.7662     
Epoch 174/3000
41578/41578 [==============================] - 0s - loss: 0.5690 - acc: 0.7667     
Epoch 175/3000
41578/41578 [==============================] - 0s - loss: 0.5688 - acc: 0.7664     
Epoch 176/3000
41578/41578 [==============================] - 0s - loss: 0.5679 - acc: 0.7683     
Epoch 177/3000
41578/4157

41578/41578 [==============================] - 0s - loss: 0.5621 - acc: 0.7684     
Epoch 250/3000
41578/41578 [==============================] - 0s - loss: 0.5611 - acc: 0.7686     
Epoch 251/3000
41578/41578 [==============================] - 0s - loss: 0.5602 - acc: 0.7697     
Epoch 252/3000
41578/41578 [==============================] - 0s - loss: 0.5609 - acc: 0.7681     
Epoch 253/3000
41578/41578 [==============================] - 0s - loss: 0.5595 - acc: 0.7704     
Epoch 254/3000
41578/41578 [==============================] - 0s - loss: 0.5598 - acc: 0.7694     
Epoch 255/3000
41578/41578 [==============================] - 0s - loss: 0.5590 - acc: 0.7692     
Epoch 256/3000
41578/41578 [==============================] - 0s - loss: 0.5593 - acc: 0.7706     
Epoch 257/3000
41578/41578 [==============================] - 0s - loss: 0.5594 - acc: 0.7704     
Epoch 258/3000
41578/41578 [==============================] - 0s - loss: 0.5588 - acc: 0.7701     
Epoch 259/3000
41578/4157

41578/41578 [==============================] - 0s - loss: 0.5554 - acc: 0.7706     
Epoch 332/3000
41578/41578 [==============================] - 0s - loss: 0.5540 - acc: 0.7706     
Epoch 333/3000
41578/41578 [==============================] - 0s - loss: 0.5549 - acc: 0.7712     
Epoch 334/3000
41578/41578 [==============================] - 0s - loss: 0.5564 - acc: 0.7710     
Epoch 335/3000
41578/41578 [==============================] - 0s - loss: 0.5537 - acc: 0.7735     
Epoch 336/3000
41578/41578 [==============================] - 0s - loss: 0.5531 - acc: 0.7717     
Epoch 337/3000
41578/41578 [==============================] - 0s - loss: 0.5532 - acc: 0.7719     
Epoch 338/3000
41578/41578 [==============================] - 0s - loss: 0.5540 - acc: 0.7719     
Epoch 339/3000
41578/41578 [==============================] - 0s - loss: 0.5543 - acc: 0.7719     
Epoch 340/3000
41578/41578 [==============================] - 0s - loss: 0.5525 - acc: 0.7722     
Epoch 341/3000
41578/4157

...

41578/41578 [==============================] - 0s - loss: 0.5060 - acc: 0.7906     
Epoch 2381/3000
41578/41578 [==============================] - 0s - loss: 0.5074 - acc: 0.7903     
Epoch 2382/3000
41578/41578 [==============================] - 0s - loss: 0.5053 - acc: 0.7918     
Epoch 2383/3000
41578/41578 [==============================] - 0s - loss: 0.5054 - acc: 0.7911     
Epoch 2384/3000
41578/41578 [==============================] - 0s - loss: 0.5047 - acc: 0.7911     
Epoch 2385/3000
41578/41578 [==============================] - 0s - loss: 0.5072 - acc: 0.7905     
Epoch 2386/3000
41578/41578 [==============================] - 0s - loss: 0.5075 - acc: 0.7906     
Epoch 2387/3000
41578/41578 [==============================] - 0s - loss: 0.5052 - acc: 0.7903     
Epoch 2388/3000
41578/41578 [==============================] - 0s - loss: 0.5055 - acc: 0.7914     
Epoch 2389/3000
41578/41578 [==============================] - 0s - loss: 0.5078 - acc: 0.7904     
Epoch 2390/3000


41578/41578 [==============================] - 0s - loss: 0.5045 - acc: 0.7915     
Epoch 2463/3000
41578/41578 [==============================] - 0s - loss: 0.5067 - acc: 0.7901     
Epoch 2464/3000
41578/41578 [==============================] - 0s - loss: 0.5036 - acc: 0.7919     
Epoch 2465/3000
41578/41578 [==============================] - 0s - loss: 0.5042 - acc: 0.7911     
Epoch 2466/3000
41578/41578 [==============================] - 0s - loss: 0.5062 - acc: 0.7904     
Epoch 2467/3000
41578/41578 [==============================] - 0s - loss: 0.5051 - acc: 0.7919     
Epoch 2468/3000
41578/41578 [==============================] - 0s - loss: 0.5068 - acc: 0.7897     
Epoch 2469/3000
41578/41578 [==============================] - 0s - loss: 0.5049 - acc: 0.7904     
Epoch 2470/3000
41578/41578 [==============================] - 0s - loss: 0.5036 - acc: 0.7921     
Epoch 2471/3000
41578/41578 [==============================] - 0s - loss: 0.5073 - acc: 0.7902     
Epoch 2472/3000


41578/41578 [==============================] - 0s - loss: 0.5056 - acc: 0.7922     
Epoch 2545/3000
41578/41578 [==============================] - 0s - loss: 0.5055 - acc: 0.7901     
Epoch 2546/3000
41578/41578 [==============================] - 0s - loss: 0.5058 - acc: 0.7909     
Epoch 2547/3000
41578/41578 [==============================] - 0s - loss: 0.5058 - acc: 0.7907     
Epoch 2548/3000
41578/41578 [==============================] - 0s - loss: 0.5053 - acc: 0.7920     
Epoch 2549/3000
41578/41578 [==============================] - 0s - loss: 0.5074 - acc: 0.7898     
Epoch 2550/3000
41578/41578 [==============================] - 0s - loss: 0.5054 - acc: 0.7904     
Epoch 2551/3000
41578/41578 [==============================] - 0s - loss: 0.5063 - acc: 0.7914     
Epoch 2552/3000
41578/41578 [==============================] - 0s - loss: 0.5066 - acc: 0.7899     
Epoch 2553/3000
41578/41578 [==============================] - 0s - loss: 0.5047 - acc: 0.7897     
Epoch 2554/3000


41578/41578 [==============================] - 1s - loss: 0.5060 - acc: 0.7907     
Epoch 2626/3000
41578/41578 [==============================] - 0s - loss: 0.5043 - acc: 0.7910     
Epoch 2627/3000
41578/41578 [==============================] - 0s - loss: 0.5045 - acc: 0.7912     
Epoch 2628/3000
41578/41578 [==============================] - 1s - loss: 0.5048 - acc: 0.7910     
Epoch 2629/3000
41578/41578 [==============================] - 1s - loss: 0.5050 - acc: 0.7917     
Epoch 2630/3000
41578/41578 [==============================] - 1s - loss: 0.5051 - acc: 0.7913     
Epoch 2631/3000
41578/41578 [==============================] - 1s - loss: 0.5075 - acc: 0.7903     
Epoch 2632/3000
41578/41578 [==============================] - 1s - loss: 0.5075 - acc: 0.7906     
Epoch 2633/3000
41578/41578 [==============================] - 1s - loss: 0.5060 - acc: 0.7925     
Epoch 2634/3000
41578/41578 [==============================] - 1s - loss: 0.5048 - acc: 0.7917     
Epoch 2635/3000


41578/41578 [==============================] - 1s - loss: 0.5041 - acc: 0.7929     
Epoch 2707/3000
41578/41578 [==============================] - 0s - loss: 0.5052 - acc: 0.7909     
Epoch 2708/3000
41578/41578 [==============================] - 0s - loss: 0.5050 - acc: 0.7921     
Epoch 2709/3000
41578/41578 [==============================] - 0s - loss: 0.5061 - acc: 0.7904     
Epoch 2710/3000
41578/41578 [==============================] - 0s - loss: 0.5055 - acc: 0.7920     
Epoch 2711/3000
41578/41578 [==============================] - 0s - loss: 0.5055 - acc: 0.7909     
Epoch 2712/3000
41578/41578 [==============================] - 0s - loss: 0.5048 - acc: 0.7906     
Epoch 2713/3000
41578/41578 [==============================] - 0s - loss: 0.5067 - acc: 0.7894     
Epoch 2714/3000
41578/41578 [==============================] - 0s - loss: 0.5040 - acc: 0.7914     
Epoch 2715/3000
41578/41578 [==============================] - 1s - loss: 0.5064 - acc: 0.7889     
Epoch 2716/3000


In [ ]:
yva_pred2 = clf.predict_classes(xva2)
sklearn.metrics.accuracy_score(y_bin.inverse_transform(yva2), yva_pred2)